In [10]:
import tensorflow as tf
import numpy as np

In [2]:
model = tf.keras.models.Sequential()

2024-03-09 15:05:10.385996: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-09 15:05:10.391124: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-09 15:05:10.391298: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10))

In [26]:
prediction = model(x_train[0].reshape(1,28,28))

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [72]:
my_loss = tf.keras.losses.SparseCategoricalCrossentropy()
my_loss([1], [0.012345, 0.176542, 0.00095754352]).numpy()

0.07264679

In [84]:
def my_backprop(target, arr, loss_fn):
    epsilon = 0.0005
    nudge = 0.012
    index = 0
    while True: 
        current = loss_fn(target, arr)
        if current - epsilon < 0:
            break
        arr[index] = arr[index] + nudge
        if loss_fn(target, arr) > current:
            arr[index] = arr[index] - 2*nudge
        index = (index + 1) % len(arr)
    print(arr)
    return arr

In [85]:
%timeit -n 1 -r 1 my_backprop([1], [0.012345, 0.176542, 0.00095754352], loss_fn)

[-4.055655000000003, 4.244541999999995, -4.055042456480003]
4.35 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [60]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0627 - accuracy: 0.9796
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0589 - accuracy: 0.9816
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0514 - accuracy: 0.9837
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0464 - accuracy: 0.9848
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0446 - accuracy: 0.9854
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0418 - accuracy: 0.9859
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0385 - accuracy: 0.9872
Epoch 8/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0367 - accuracy: 0.9878
Epoch 9/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0346 - accuracy: 0.9882
Epoch 10/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.031

In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0758 - accuracy: 0.9776 - 428ms/epoch - 1ms/step


[0.0758436843752861, 0.9775999784469604]

In [86]:
model2 = tf.keras.models.Sequential()

In [130]:
model2.add(tf.keras.layers.Conv2D(32, 3, input_shape=(28, 28, 1)))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(10))

In [133]:
model2.compile(optimizer='rmsprop', loss=loss_fn, metrics=['accuracy'])

In [142]:
tf.config.run_functions_eagerly(True)

In [146]:
model2.evaluate(x_test.reshape(10000, 28, 28, 1), y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.2953 - accuracy: 0.9173


[0.29531946778297424, 0.9172999858856201]

In [147]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           multiple                  320       
                                                                 
 flatten_5 (Flatten)         multiple                  0         
                                                                 
 dense_6 (Dense)             multiple                  216330    
                                                                 
Total params: 216650 (846.29 KB)
Trainable params: 216650 (846.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [151]:
model3 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(64, 3, input_shape=(28, 28, 1)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10)])


In [153]:
model3.compile(optimizer='rmsprop', loss=loss_fn, metrics=['accuracy'])
model3.fit(x_train.reshape(60000, 28, 28, 1), y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 29s 15ms/step - loss: 0.3313 - accuracy: 0.9067
Epoch 2/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.2999 - accuracy: 0.9166
Epoch 3/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.2890 - accuracy: 0.9207
Epoch 4/10
1875/1875 [==============================] - 33s 18ms/step - loss: 0.2828 - accuracy: 0.9216
Epoch 5/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.2779 - accuracy: 0.9232
Epoch 6/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.2758 - accuracy: 0.9239
Epoch 7/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.2722 - accuracy: 0.9244
Epoch 8/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.2703 - accuracy: 0.9257
Epoch 9/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.2693 - accuracy: 0.9255
Epoch 10/10
1875/1875 [==============================] - 35s 19m

In [154]:
model4 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(64, 3, input_shape=(28, 28, 1), activation='relu'),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10)])
model4.compile(optimizer='rmsprop', loss=loss_fn, metrics=['accuracy'])
model4.fit(x_train.reshape(60000, 28, 28, 1), y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.1645 - accuracy: 0.9513
Epoch 2/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0703 - accuracy: 0.9799
Epoch 3/10
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0538 - accuracy: 0.9845
Epoch 4/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0449 - accuracy: 0.9872
Epoch 5/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0376 - accuracy: 0.9888
Epoch 6/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0328 - accuracy: 0.9905
Epoch 7/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0278 - accuracy: 0.9922
Epoch 8/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0245 - accuracy: 0.9932
Epoch 9/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0216 - accuracy: 0.9940
Epoch 10/10
1875/1875 [==============================] - 36s 19m

In [15]:
model5 = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, 3, input_shape=(28, 28, 1), activation='relu'),
tf.keras.layers.Conv2D(32, 3, input_shape=(28, 28, 1), activation='relu'),
tf.keras.layers.MaxPooling2D(strides=2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10)])
model5.compile(optimizer='rmsprop', loss=loss_fn, metrics=['accuracy'])
model5.fit(x_train.reshape(60000, 28, 28, 1), y_train, epochs=6)

Epoch 1/6


2024-03-11 12:22:06.200215: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-11 12:22:06.848890: I external/local_xla/xla/service/service.cc:168] XLA service 0x7bdffff54aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 12:22:06.848912: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-03-11 12:22:06.875809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710139927.098307  260771 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 6s 2ms/step - loss: 0.1537 - accuracy: 0.9547
Epoch 2/6
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0612 - accuracy: 0.9819
Epoch 3/6
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0481 - accuracy: 0.9859
Epoch 4/6
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0398 - accuracy: 0.9884
Epoch 5/6
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0355 - accuracy: 0.9900
Epoch 6/6
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0314 - accuracy: 0.9912


In [160]:
np.argmax(tf.nn.softmax(model5.predict(x_test[:1])))

1/1 [==============================] - 0s 14ms/step


7

In [161]:
y_test[:1]

array([7], dtype=uint8)

In [163]:
model5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 26, 26, 16)        160       
                                                                 
 conv2d_14 (Conv2D)          (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 flatten_9 (Flatten)         (None, 4608)              0         
                                                                 
 dense_10 (Dense)            (None, 10)                46090     
                                                                 
Total params: 50890 (198.79 KB)
Trainable params: 50890 (198.79 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [25]:
model5.save("model5-mnist.keras")

In [16]:
tf.keras.models.save_model(model5, "saved_model")

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [5]:
model_saved = tf.keras.models.load_model("saved_model")

In [ ]:
## QAT for MNIST

In [ ]:
import tensorflow_model_optimization as tfmot
quantized_model = tfmot.quantization.keras.quantize_model(model_saved)

In [27]:
quantized_model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [30]:
train_images_subset = x_train # out of 60000
train_labels_subset = y_train

In [32]:
quantized_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=1, validation_split=0.1)

108/108 [==============================] - 15s 140ms/step - loss: 0.0356 - accuracy: 0.9911 - val_loss: 0.0307 - val_accuracy: 0.9922


In [33]:
model_saved.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.9874


[0.03832940384745598, 0.9873999953269958]

In [34]:
quantized_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0370 - accuracy: 0.9889


[0.03696480020880699, 0.9889000058174133]

In [22]:
## QAT to Quantized/Serialized tflite model

In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2o0w2sn6/assets


INFO:tensorflow:Assets written to: /tmp/tmp2o0w2sn6/assets
/home/metal/dev/misc/python/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-03-15 16:28:44.135760: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-15 16:28:44.135786: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-15 16:28:44.136061: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp2o0w2sn6
2024-03-15 16:28:44.137633: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-15 16:28:44.137645: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp2o0w2sn6
2024-03-15 16:28:44.140877: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization p

In [25]:
with open("model5-quantized.tflite", 'wb') as f:
  f.write(quantized_tflite_model)